I am going to produce the InventoryLedger.csv here

[Read what each table has](../../Data/Processed/tables.txt)

<div>
  <img src="../../Documentation/image.png" width="1000">
</div>

# --------------------------------------------------------
Step 1: Load CSV Files

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [2]:
# Load the csv files
inventory_df = pd.read_csv('../../Data/Processed/Inventory.csv')
opening_stock_df = pd.read_csv('../../Data/Processed/OpeningStock.csv')
purchases_df = pd.read_csv('../../Data/Processed/Purchases.csv')
sales_df = pd.read_csv('../../Data/Processed/Sales.csv')
stores_df = pd.read_csv('../../Data/Processed/Stores.csv')

# --------------------------------------------------------

Step 2: Preprocess and Format Opening Stock

In [3]:
opening_stock_ledger = opening_stock_df.copy()
opening_stock_ledger['TranType'] = 'StockTake'
opening_stock_ledger['Quantity'] = opening_stock_ledger['onHand']  # Positive for stock take
opening_stock_ledger['OrderDate'] = opening_stock_ledger['startDate']  # Using startDate as OrderDate
opening_stock_ledger['StockDate'] = opening_stock_ledger['startDate']  # Using startDate as StockDate
opening_stock_ledger.rename(columns={'StoreID': 'StoreID', 'ItemId': 'ItemID'}, inplace=True)

# Drop the uneccesary column
opening_stock_ledger.drop(columns=['onHand'], inplace=True)
opening_stock_ledger.drop(columns=['startDate'], inplace=True)

#Order the ledger columns
opening_stock_ledger = opening_stock_ledger[['StoreID', 'ItemID', 'TranType', 'Quantity', 'OrderDate', 'StockDate']]

opening_stock_ledger.head()

,StoreID,ItemID,TranType,Quantity,OrderDate,StockDate
0,1,58,StockTake,8,2015-12-31,2015-12-31
1,1,60,StockTake,7,2015-12-31,2015-12-31
2,1,62,StockTake,6,2015-12-31,2015-12-31
3,1,63,StockTake,3,2015-12-31,2015-12-31
4,1,72,StockTake,6,2015-12-31,2015-12-31


# --------------------------------------------------------

Step 3: Preprocess and Format Purchases

In [4]:
purchases_ledger = purchases_df.copy()
purchases_ledger['TranType'] = 'Purchase'
purchases_ledger['Quantity'] = purchases_ledger['Quantity']  # Positive for purchases
purchases_ledger.rename(columns={'StoreID': 'StoreID', 'PODate': 'OrderDate', 'ReceivingDate': 'StockDate', 'ItemId': 'ItemID'}, inplace=True)

#Order the ledger columns
purchases_ledger = purchases_ledger[['StoreID', 'ItemID', 'TranType', 'Quantity', 'OrderDate', 'StockDate']]

purchases_ledger.head()

,StoreID,ItemID,TranType,Quantity,OrderDate,StockDate
0,1,5255,Purchase,6,2015-12-22,2016-01-01
1,1,8358,Purchase,12,2015-12-20,2016-01-01
2,1,4233,Purchase,23,2015-12-20,2016-01-01
3,1,3830,Purchase,6,2015-12-20,2016-01-01
4,1,4670,Purchase,11,2015-12-20,2016-01-01


# --------------------------------------------------------

Step 4: Preprocess and Format Sales

In [5]:
sales_ledger = sales_df.copy()
sales_ledger['TranType'] = 'Sale'
sales_ledger['Quantity'] = -sales_ledger['SalesQuantity']  # Negative for sales
sales_ledger['OrderDate'] = sales_ledger['SalesDate']  # Using SalesDate as OrderDate
sales_ledger['StockDate'] = sales_ledger['SalesDate']  # Using SalesDate as StockDate
sales_ledger.rename(columns={'StoreID': 'StoreID', 'ItemId': 'ItemID'}, inplace=True)

# Drop the uneccesary column
sales_ledger.drop(['SalesQuantity', 'SalesDate'], axis=1, inplace=True)

#Order the ledger columns
sales_ledger = sales_ledger[['StoreID', 'ItemID', 'TranType', 'Quantity', 'OrderDate', 'StockDate']]

sales_ledger.head()

,StoreID,ItemID,TranType,Quantity,OrderDate,StockDate
0,1,1004,Sale,-1,1/1/2016,1/1/2016
1,1,1004,Sale,-2,1/2/2016,1/2/2016
2,1,1004,Sale,-1,1/3/2016,1/3/2016
3,1,1004,Sale,-1,1/8/2016,1/8/2016
4,1,1005,Sale,-2,1/9/2016,1/9/2016


# --------------------------------------------------------

Step 5: Combine All Transactions

In [6]:
combined_ledger = pd.concat([opening_stock_ledger, purchases_ledger, sales_ledger], ignore_index=True)

combined_ledger.head()

,StoreID,ItemID,TranType,Quantity,OrderDate,StockDate
0,1,58,StockTake,8,2015-12-31,2015-12-31
1,1,60,StockTake,7,2015-12-31,2015-12-31
2,1,62,StockTake,6,2015-12-31,2015-12-31
3,1,63,StockTake,3,2015-12-31,2015-12-31
4,1,72,StockTake,6,2015-12-31,2015-12-31


# --------------------------------------------------------

Step 6: add Transaction IDs, Inventory and Store Info

In [11]:
# Add transaction IDs
combined_ledger['TranID'] = combined_ledger.index + 1

# Add store location from stores_df
combined_ledger = pd.merge(combined_ledger, stores_df[['StoreID', 'Location']], 
                         on='StoreID', how='left')
combined_ledger = combined_ledger.rename(columns={'Location': 'storelocation'})

# Add item description from inventory_df
combined_ledger = pd.merge(combined_ledger, inventory_df[['ItemID', 'Description']], 
                         on='ItemID', how='left')

combined_ledger.head()

,StoreID,ItemID,TranType,Quantity,OrderDate,StockDate,TranID,StoreLocation,storelocation,storelocation,Description_x,storelocation,Description_y
0,1,58,StockTake,8,2015-12-31,2015-12-31,1,HARDERSFIELD,HARDERSFIELD,HARDERSFIELD,Gekkeikan Black & Gold Sake,HARDERSFIELD,Gekkeikan Black & Gold Sake
1,1,60,StockTake,7,2015-12-31,2015-12-31,2,HARDERSFIELD,HARDERSFIELD,HARDERSFIELD,Canadian Club 1858 VAP,HARDERSFIELD,Canadian Club 1858 VAP
2,1,62,StockTake,6,2015-12-31,2015-12-31,3,HARDERSFIELD,HARDERSFIELD,HARDERSFIELD,Herradura Silver Tequila,HARDERSFIELD,Herradura Silver Tequila
3,1,63,StockTake,3,2015-12-31,2015-12-31,4,HARDERSFIELD,HARDERSFIELD,HARDERSFIELD,Herradura Reposado Tequila,HARDERSFIELD,Herradura Reposado Tequila
4,1,72,StockTake,6,2015-12-31,2015-12-31,5,HARDERSFIELD,HARDERSFIELD,HARDERSFIELD,No. 3 London Dry Gin,HARDERSFIELD,No. 3 London Dry Gin


# --------------------------------------------------------

Step 6: add Inventory and Store Info

In [ ]:
combined_ledger = pd.merge(combined_ledger, inventory_df[['ItemID', 'Description']], on='ItemID', how='left')

combined_ledger = pd.merge(combined_ledger, stores_df[['StoreID', 'Location']], left_on='StoreID', right_on='StoreID', how='left')
combined_ledger.rename(columns={'Location': 'StoreLocation'}, inplace=True)


combined_ledger.head()  

,StoreID,ItemID,TranType,Quantity,OrderDate,StockDate,Description,StoreLocation
0,1,58,StockTake,8,2015-12-31,2015-12-31,Gekkeikan Black & Gold Sake,HARDERSFIELD
1,1,60,StockTake,7,2015-12-31,2015-12-31,Canadian Club 1858 VAP,HARDERSFIELD
2,1,62,StockTake,6,2015-12-31,2015-12-31,Herradura Silver Tequila,HARDERSFIELD
3,1,63,StockTake,3,2015-12-31,2015-12-31,Herradura Reposado Tequila,HARDERSFIELD
4,1,72,StockTake,6,2015-12-31,2015-12-31,No. 3 London Dry Gin,HARDERSFIELD


# --------------------------------------------------------

Step 6: add Inventory and Store Info

# --------------------------------------------------------

Step 5: Combine All Transactions

# --------------------------------------------------------

Step 5: Combine All Transactions